In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [14]:
df_industries = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/industries_indexes.csv',encoding = 'iso-8859-1')
df_industries['Industry Trade Dependency'] = df_industries['Industry Trade Dependency']
df_industries['Import-Export Imbalance'] = df_industries['Import-Export Imbalance']
df_industries['Import Growth'] = df_industries['Import Growth']
df_industries['Export Growth'] = df_industries['Export Growth']

In [15]:
df_industries

,Unnamed: 0,Time,Area,Industry,Export,Import,Industry Trade Dependency,Import-Export Imbalance,Export Growth,Import Growth,Export Growth (Absolute),Import Growth (Absolute)
0,0,2015,MK01,A,1.267644e+08,2.509054e+08,0.457862,32.870242,0.000000,0.000000,0.000000e+00,0.000000e+00
1,1,2015,MK01,B,7.414043e+06,1.467465e+07,0.026779,32.870242,0.000000,0.000000,0.000000e+00,0.000000e+00
2,2,2015,MK01,C,1.521114e+09,3.010749e+09,5.494135,32.870242,0.000000,0.000000,0.000000e+00,0.000000e+00
3,3,2015,MK01,D,1.516152e+08,3.000927e+08,0.547621,32.870242,0.000000,0.000000,0.000000e+00,0.000000e+00
4,4,2015,MK01,F,1.209391e+09,2.393754e+09,4.368218,32.870242,0.000000,0.000000,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
2047,2047,2020,MK21,O,6.757047e+06,2.456386e+07,2.606209,56.852792,-21.732368,-25.651533,-1.876212e+06,-8.474964e+06
2048,2048,2020,MK21,P,4.881392e+06,1.774530e+07,1.882764,56.852792,-19.052176,-23.105550,-1.148902e+06,-5.332177e+06
2049,2049,2020,MK21,Q,1.273570e+07,4.629801e+07,4.912188,56.852792,-16.181137,-20.378274,-2.458612e+06,-1.184945e+07
2050,2050,2020,MK21,R,4.337452e+06,1.576792e+07,1.672965,56.852792,-17.928649,-22.038281,-9.475250e+05,-4.457287e+06


In [16]:
regions = df_industries['Area'].unique().tolist()
industries = df_industries['Industry'].unique().tolist()
years = np.unique(df_industries['Time'])
future_years = [2021, 2022, 2023]
time = [2015,2016,2017,2018,2019,2020,2021,2022,2023]

In [17]:
# Exports growth by region and industries
all_exports_growth = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Export Growth']
            export_growth_model = LinearRegression()
            export_growth_model.fit(X_train, y_train)
            # Predictions for test set
            export_growth_pred = export_growth_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Export Growth'], export_growth_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = export_growth_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_export_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            export_growth_pred = best_model.predict(X_future)
            future_preds_export_growth.append(export_growth_pred[0])
        # Saving results
        all_exports_growth.append(filtered_df['Export Growth'].tolist() + future_preds_export_growth)

# Saving results to csv
df_exports_growth = pd.DataFrame()
df_exports_growth['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_exports_growth[column_name] = all_exports_growth[index]
        index += 1
df_exports_growth.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports_growth_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\3330239465.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exports[column_name] = all_exports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\3330239465.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exports[column_name] = all_exports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\3330239465.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

In [18]:
# Do the same for import growths
all_imports_growth = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Import Growth']
            import_growth_model = LinearRegression()
            import_growth_model.fit(X_train, y_train)
            import_growth_pred = import_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Import Growth'], import_growth_pred)
            if mse < best_mse:
                best_model = import_growth_model
                best_mse = mse
        future_preds_import_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            import_growth_pred = best_model.predict(X_future)
            future_preds_import_growth.append(import_growth_pred[0])
        all_imports_growth.append(filtered_df['Import Growth'].tolist() + future_preds_import_growth)
df_imports_growth = pd.DataFrame()
df_imports_growth['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_imports_growth[column_name] = all_imports_growth[index]
        index += 1
df_imports_growth.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports_growth_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\1548896995.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imports[column_name] = all_imports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\1548896995.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imports[column_name] = all_imports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\1548896995.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons

In [19]:
# Do the same for trade dependencies
all_trades = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Industry Trade Dependency']
            trade_growth_model = LinearRegression()
            trade_growth_model.fit(X_train, y_train)
            trade_growth_pred = trade_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Industry Trade Dependency'], trade_growth_pred)
            if mse < best_mse:
                best_model = trade_growth_model
                best_mse = mse
        future_preds_trade_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            trade_growth_pred = best_model.predict(X_future)
            future_preds_trade_growth.append(trade_growth_pred[0])
        all_trades.append(filtered_df['Industry Trade Dependency'].tolist() + future_preds_trade_growth)
df_trades = pd.DataFrame()
df_trades['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_trades[column_name] = all_trades[index]
        index += 1
df_trades.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/trade_dependencies_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2353735164.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_trades[column_name] = all_trades[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2353735164.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_trades[column_name] = all_trades[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2353735164.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

In [20]:
# Do the same for import-export imbalance
all_imbas = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Import-Export Imbalance']
            imba_growth_model = LinearRegression()
            imba_growth_model.fit(X_train, y_train)
            imba_growth_pred = imba_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Import-Export Imbalance'], imba_growth_pred)
            if mse < best_mse:
                best_model = imba_growth_model
                best_mse = mse
        future_preds_imba_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            imba_growth_pred = best_model.predict(X_future)
            future_preds_imba_growth.append(imba_growth_pred[0])
        all_imbas.append(filtered_df['Import-Export Imbalance'].tolist() + future_preds_imba_growth)
df_imbas = pd.DataFrame()
df_imbas['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_imbas[column_name] = all_imbas[index]
        index += 1
df_imbas.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imbalances_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2188531957.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imbas[column_name] = all_imbas[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2188531957.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imbas[column_name] = all_imbas[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2188531957.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

In [ ]:
# Do the same for imports growth (abs)
all_exports_growth_abs = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Export Growth (Absolute)']
            export_growth_abs_model = LinearRegression()
            export_growth_abs_model.fit(X_train, y_train)
            export_growth_abs_pred = export_growth_abs_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Export Growth (Absolute)'], export_growth_abs_pred)
            if mse < best_mse:
                best_model = export_growth_abs_model
                best_mse = mse
        future_preds_export_growth_abs = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            export_growth_abs_pred = best_model.predict(X_future)
            future_preds_export_growth_abs.append(export_growth_abs_pred[0])
        all_exports_growth_abs.append(filtered_df['Export Growth (Absolute)'].tolist() + future_preds_export_growth_abs)
df_exports_growth_abs = pd.DataFrame()
df_exports_growth_abs['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_exports_growth_abs[column_name] = all_exports_growth_abs[index]
        index += 1
df_exports_growth_abs.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports_growth_abs_industry.csv')

In [21]:
# Do the same for imports growth (abs)
all_imports_growth_abs = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Import Growth (Absolute)']
            import_growth_abs_model = LinearRegression()
            import_growth_abs_model.fit(X_train, y_train)
            import_growth_abs_pred = import_growth_abs_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Import Growth (Absolute)'], import_growth_abs_pred)
            if mse < best_mse:
                best_model = import_growth_abs_model
                best_mse = mse
        future_preds_import_growth_abs = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            import_growth_abs_pred = best_model.predict(X_future)
            future_preds_import_growth_abs.append(import_growth_abs_pred[0])
        all_imports_growth_abs.append(filtered_df['Import Growth (Absolute)'].tolist() + future_preds_import_growth_abs)
df_imports_growth_abs = pd.DataFrame()
df_imports_growth_abs['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_imports_growth_abs[column_name] = all_imports_growth_abs[index]
        index += 1
df_imports_growth_abs.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports_growth_abs_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2620907129.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imports_abs[column_name] = all_imports_abs[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2620907129.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imports_abs[column_name] = all_imports_abs[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2620907129.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [11]:
# Do the same for exports
all_exp = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Export']
            exp_model = LinearRegression()
            exp_model.fit(X_train, y_train)
            exp_pred = exp_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Export'], exp_pred)
            if mse < best_mse:
                best_model = exp_model
                best_mse = mse
        future_preds_exp = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            exp_pred = best_model.predict(X_future)
            future_preds_exp.append(exp_pred[0])
        all_exp.append(filtered_df['Export'].tolist() + future_preds_exp)
df_exp = pd.DataFrame()
df_exp['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_exp[column_name] = all_exp[index]
        index += 1
df_exp.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2227804190.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exp[column_name] = all_exp[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2227804190.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exp[column_name] = all_exp[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\2227804190.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [22]:
# Do the same for imports
all_imp = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Import']
            imp_model = LinearRegression()
            imp_model.fit(X_train, y_train)
            imp_pred = imp_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Import'], imp_pred)
            if mse < best_mse:
                best_model = imp_model
                best_mse = mse
        future_preds_imp = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            imp_pred = best_model.predict(X_future)
            future_preds_imp.append(imp_pred[0])
        all_imp.append(filtered_df['Import'].tolist() + future_preds_imp)
df_imp = pd.DataFrame()
df_imp['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_imp[column_name] = all_imp[index]
        index += 1
df_imp.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\86128913.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imp[column_name] = all_imp[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\86128913.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imp[column_name] = all_imp[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_9336\86128913.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum